In [1]:
import duckdb

In [2]:
duckdb.sql('INSTALL httpfs')
duckdb.sql('LOAD httpfs')
duckdb.sql("FORCE INSTALL spatial FROM 'http://nightly-extensions.duckdb.org';")
duckdb.sql('LOAD spatial')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
prefix = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-open-buildings/geoparquet"
partitions = "by_country"
country_iso = "IND"

In [ ]:
import numpy as np
from tqdm import tqdm
for i in tqdm(np.arange(0,520127455,10000000)):
    output_file = 'buildings_{}.geojson'.format(int(i/10000000))
    duckdb.sql(f"COPY (SELECT ST_Centroid(ST_GeomFromWKB(geometry)) AS centroid, area_in_meters FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet') LIMIT 10000 OFFSET {i}) TO '{output_file}' WITH  (FORMAT GDAL, DRIVER 'GeoJSON');")

  0%|          | 0/53 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  2%|▏         | 1/53 [00:21<18:32, 21.40s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
output_file = 'check.geojson'
duckdb.sql(f"COPY (SELECT ST_Centroid(ST_GeomFromWKB(geometry)) AS centroid, area_in_meters FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet') LIMIT 10) TO '{output_file}' WITH  (FORMAT GDAL, DRIVER 'GeoJSON');")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:


# Use single file
#duckdb.sql(f"SELECT * FROM '{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet'").show()
duckdb.sql(f"SELECT ST_Centroid(ST_GeomFromWKB(geometry)), area_in_meters FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet') LIMIT 10").show()

#duckdb.sql(f"CREATE TABLE ind_buildings AS SELECT * FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/*.parquet')")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────────────┬────────────────────┐
│    st_centroid(st_geomfromwkb(geometry))     │   area_in_meters   │
│                   geometry                   │       double       │
├──────────────────────────────────────────────┼────────────────────┤
│ POINT (92.74675637483597 11.618450683198503) │ 11.606829295371362 │
│ POINT (92.71531756178324 11.67963819573042)  │  650.6862792975276 │
│ POINT (92.68356792593657 22.28837828583811)  │ 24.871156367986874 │
│ POINT (92.68091186520215 11.604189169151153) │  24.40779338300694 │
│ POINT (92.65175248418431 22.281230688937267) │  31.83381183719124 │
│ POINT (92.72015348035085 11.65292237234046)  │  14.42021149383084 │
│ POINT (90.00002386152127 25.302263225889302) │ 21.433733484317933 │
│ POINT (92.63796764473217 22.131304296193633) │ 41.804944354344634 │
│ POINT (92.66767818689348 22.183248964458564) │  20.12778096908015 │
│ POINT (92.63553601859496 22.356731629773876) │ 22.081087010413913 │
├───────────────────

In [6]:
duckdb.sql(f"SELECT COUNT(*) FROM parquet_scan('{prefix}/{partitions}/country_iso={country_iso}/{country_iso}.parquet')").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    520127455 │
└──────────────┘



In [5]:
duckdb.sql(f"SELECT * FROM lso_buildings LIMIT 10").show()

┌────────────────────────────────────────────┬─────────────┬───────────┬────────────┬────────────────────┬─────────────┐
│                  geometry                  │ boundary_id │ bf_source │ confidence │   area_in_meters   │ country_iso │
│                    blob                    │    int64    │  varchar  │   double   │       double       │   varchar   │
├────────────────────────────────────────────┼─────────────┼───────────┼────────────┼────────────────────┼─────────────┤
│ \x01\x03\x00\x00\x00\x01\x00\x00\x00\x05…  │          81 │ microsoft │       NULL │ 16.642731770012922 │ LSO         │
│ \x01\x03\x00\x00\x00\x01\x00\x00\x00\x05…  │          81 │ microsoft │       NULL │ 25.228388299419247 │ LSO         │
│ \x01\x03\x00\x00\x00\x01\x00\x00\x00\x05…  │          81 │ microsoft │       NULL │ 23.074620278788217 │ LSO         │
│ \x01\x03\x00\x00\x00\x01\x00\x00\x00\x05…  │          81 │ microsoft │       NULL │ 41.838311520336326 │ LSO         │
│ \x01\x03\x00\x00\x00\x01\x00\x

In [6]:
duckdb.sql(f"SELECT bf_source, COUNT(*) AS buildings_count FROM lso_buildings GROUP BY bf_source;").show()

┌───────────┬─────────────────┐
│ bf_source │ buildings_count │
│  varchar  │      int64      │
├───────────┼─────────────────┤
│ google    │         1394189 │
│ microsoft │          151722 │
└───────────┴─────────────────┘



In [9]:
duckdb.sql(f"SELECT ST_Centroid(ST_GeomFromWKB(geometry)) AS Centroid FROM lso_buildings LIMIT 10").show()

┌────────────────────────────────────────────────┐
│                    Centroid                    │
│                    geometry                    │
├────────────────────────────────────────────────┤
│ POINT (28.075053834908836 -30.238157536375958) │
│ POINT (28.26500512745651 -30.20128607386714)   │
│ POINT (28.138475844863034 -30.224001799244114) │
│ POINT (27.80845044855237 -30.225979266383238)  │
│ POINT (27.542520976589874 -29.85507790192917)  │
│ POINT (27.715311320589844 -30.402095904104623) │
│ POINT (27.641707514237215 -30.392490552626775) │
│ POINT (27.458112422751537 -30.048585938939446) │
│ POINT (27.73843831888474 -30.076922704111638)  │
│ POINT (27.367864626259102 -29.92227828758444)  │
├────────────────────────────────────────────────┤
│                    10 rows                     │
└────────────────────────────────────────────────┘



In [10]:
output_file = "check.geojson"
duckdb.sql(f"COPY (SELECT ST_Centroid(ST_GeomFromWKB(geometry)) AS Centroid FROM lso_buildings LIMIT 10) TO '{output_file}' WITH  (FORMAT GDAL, DRIVER 'GeoJSON');")